In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers

In [20]:
dataset_dir = "Datasets/winequality-red.csv"

In [21]:
df = pd.read_csv(dataset_dir, sep = ";", index_col=None)
df = df.fillna(df.mean())
df['quality'] = (df['quality'] >= 6).astype(int)
label_column = df.pop('quality')
label_column = np.asarray(label_column)
df.insert(0, 'label', label_column)
df

,label,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,0,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,0,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,1,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,1,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,1,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,0,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [22]:
df.isnull().values.any()

False

In [23]:
df.to_csv("./Datasets/red_wine_binary.csv")

In [5]:
df = df.sample(frac=1, random_state=41)

In [6]:
col_num = len(df.columns)
row_num = len(df.index)
print("Number of columns is: {} and number of rows is: {}".format(col_num, row_num))

Number of columns is: 12 and number of rows is: 1599


In [7]:
trainx = df.iloc[:int(0.8*row_num), 1:(col_num)]
trainy = df.iloc[:int(0.8*row_num), 0]

testx = df.iloc[int(0.8*row_num):, 1:(col_num)]
testy = df.iloc[int(0.8*row_num):, 0]

In [8]:
scaler = StandardScaler()
trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx), columns = trainx.columns, index = trainx.index)
textx_scaled = pd.DataFrame(scaler.transform(testx), columns = testx.columns, index = testx.index)

In [9]:
def get_MICS_model(inp_size, drop_out, hidden_num = 3, hidden_size=256):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    h = keras.layers.BatchNormalization()(h)
    for hidden in range(hidden_num):
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
        h = keras.layers.Dropout(drop_out)(h) 
        h = keras.layers.BatchNormalization()(h)

    outputs = keras.layers.Dense(1, activation="sigmoid")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [10]:
def get_MICS_model(inp_size, drop_out, hidden_num = 3, hidden_size=256):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    h = keras.layers.BatchNormalization()(h)
    h = keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h)
    h = keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h)    
    h = keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h) 
    h = keras.layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
    h = keras.layers.Dropout(drop_out)(h) 
    h = keras.layers.BatchNormalization()(h) 
    
    outputs = keras.layers.Dense(1, activation="sigmoid")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [11]:
df["label"].value_counts()

1    855
0    744
Name: label, dtype: int64

In [14]:
inp_size = len(trainx.columns)
MICS_model = get_MICS_model(inp_size, drop_out = 0.25)
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30), 
        keras.callbacks.ReduceLROnPlateau("val_loss", factor = np.sqrt(1/10), patience=15,
                                         verbose = 2, mode = "auto", 
                                          min_lr = 1e-5)]
MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(), metrics = ["accuracy"])
history = MICS_model.fit(x = [trainx_scaled], y = trainy.values,  
                         validation_data = ([textx_scaled], testy.values),
                         epochs=300, batch_size = 32, callbacks=callback)
training_val_accuracy = history.history["val_accuracy"]
best_row_index = np.argmax(training_val_accuracy)
best_val_accuracy = training_val_accuracy[best_row_index]
best_val_accuracy
#min_losses3[c].append(best_val_loss)
#i = i + 1

Epoch 1/300
40/40 [==============================] - 0s 9ms/step - loss: 2.3738 - accuracy: 0.6474 - val_loss: 2.2644 - val_accuracy: 0.7437
Epoch 2/300
40/40 [==============================] - 0s 5ms/step - loss: 2.0957 - accuracy: 0.7013 - val_loss: 1.8134 - val_accuracy: 0.7281
Epoch 3/300
40/40 [==============================] - 0s 5ms/step - loss: 1.5917 - accuracy: 0.7217 - val_loss: 1.4029 - val_accuracy: 0.7531
Epoch 4/300
40/40 [==============================] - 0s 5ms/step - loss: 1.2586 - accuracy: 0.7342 - val_loss: 1.2270 - val_accuracy: 0.7656
Epoch 5/300
40/40 [==============================] - 0s 5ms/step - loss: 1.0978 - accuracy: 0.7123 - val_loss: 1.0700 - val_accuracy: 0.7000
Epoch 6/300
40/40 [==============================] - 0s 5ms/step - loss: 0.9651 - accuracy: 0.7279 - val_loss: 1.0065 - val_accuracy: 0.7594
Epoch 7/300
40/40 [==============================] - 0s 6ms/step - loss: 0.9325 - accuracy: 0.7115 - val_loss: 0.9709 - val_accuracy: 0.6500
Epoch 8/300
4

0.8218749761581421

In [ ]:
df["label"].value_counts()